<a href="https://colab.research.google.com/github/malaika02/sentiment_analysis/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.layers import *
from keras.models import Sequential
import re

In [2]:
from tensorflow.keras.callbacks import EarlyStopping

In [26]:
# configuring the path of Kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis

Dataset URL: https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis
License(s): CC0-1.0
  0% 0.00/1.99M [00:00<?, ?B/s]
100% 1.99M/1.99M [00:00<00:00, 198MB/s]


In [5]:
from zipfile import ZipFile
dataset = '/content/twitter-entity-sentiment-analysis.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('dataset extracted')

dataset extracted


In [27]:
df = pd.read_csv('/content/twitter_training.csv')

In [ ]:
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [28]:
df.columns = ['id','place','label','text']

In [ ]:
df.head()

,id,place,label,text
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [29]:
df.drop(columns=['id','place'],inplace=True)

In [ ]:
df.head()

,label,text
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [30]:
df['label']=df['label'].map({'Positive':1,'Negative':0})

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74681 entries, 0 to 74680
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   43373 non-null  float64
 1   text    73995 non-null  object 
dtypes: float64(1), object(1)
memory usage: 1.1+ MB


In [31]:
df= df[(df['label']==0) | (df['label']==1)]

In [32]:
positive = print(len(df[df['label']==1]))
negative = len(df[df['label']==0])

20831


22542

In [11]:
df.shape

(43373, 2)

In [12]:
df.head()

,label,text
0,1.0,I am coming to the borders and I will kill you...
1,1.0,im getting on borderlands and i will kill you ...
2,1.0,im coming on borderlands and i will murder you...
3,1.0,im getting on borderlands 2 and i will murder ...
4,1.0,im getting into borderlands and i can murder y...


In [17]:
from tensorflow.keras.layers import TextVectorization

In [33]:
df = df.dropna()

In [34]:
df.isna().sum()

,0
label,0
text,0


In [37]:
X = df['text']
y = df['label']

In [38]:
# Split the data into training and testing sets
X_train,X_test,y_train,y_test= train_test_split(X,y, test_size=0.2, random_state=42)

In [39]:
X_train.shape

(34409,)

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
# remove stopwords
import re
import string
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [35]:
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and digits
    text = re.sub(r'\d+',' ', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(f"[{string.punctuation}]",' ',text)
    return text



In [36]:
df['text'] = df['text'].apply(clean_text)

In [40]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [41]:
tokenizer = Tokenizer()

In [42]:
tokenizer.fit_on_texts(X_train)

In [43]:
len(tokenizer.word_index)

18391

In [44]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [45]:
lenn = 0
for ele in X_train:
  lenn = max(lenn,len(ele))

In [46]:
lenn   #padded_sequence

166

In [47]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train = pad_sequences(X_train,maxlen=166,padding='post')
X_test = pad_sequences(X_test,maxlen=166,padding='post')

In [48]:
from tensorflow.keras.regularizers import l2

In [68]:
model = Sequential()

model.add(Embedding(input_dim=18392,output_dim=32))
model.add(Bidirectional(LSTM(120,kernel_regularizer=l2(0.001))))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid',kernel_regularizer=l2(0.001)))
model.build(input_shape=(None, 166))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ (None, 166, 32)             │         588,544 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_3 (Bidirectional)      │ (None, 240)                 │         146,880 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 240)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             241 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 735,665 (2.81 MB)

 Trainable params: 735,665 (2.81 MB)

 Non-trainable params: 0 (0.00 B)

In [69]:
early_stop = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True,verbose=1)

In [70]:
model.fit(X_train,y_train,epochs=50,validation_data=(X_test,y_test),callbacks=[early_stop],batch_size=32)

Epoch 1/50
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 22s 18ms/step - accuracy: 0.7141 - loss: 0.5803 - val_accuracy: 0.8866 - val_loss: 0.3040
Epoch 2/50
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 19s 17ms/step - accuracy: 0.9155 - loss: 0.2347 - val_accuracy: 0.9177 - val_loss: 0.2231
Epoch 3/50
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - accuracy: 0.9428 - loss: 0.1683 - val_accuracy: 0.9233 - val_loss: 0.2082
Epoch 4/50
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - accuracy: 0.9490 - loss: 0.1483 - val_accuracy: 0.9257 - val_loss: 0.2019
Epoch 5/50
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 21s 17ms/step - accuracy: 0.9571 - loss: 0.1270 - val_accuracy: 0.9291 - val_loss: 0.1934
Epoch 6/50
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - accuracy: 0.9593 - loss: 0.1184 - val_accuracy: 0.9272 - val_loss: 0.1923
Epoch 7/50
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 20s 17ms/step - accuracy: 0.9601 - loss: 0.1116 - val_accuracy: 0.9258 - val_loss: 0.2005
Epoch 8/50
1076/1076 ━━━━━━━━━━━━━━━━━━━━ 21s 18ms/step - accuracy: 0.9644 -

In [55]:
from tensorflow.keras.models import load_model

# Load the entire model
model = model.save('sentiment_analysis_model.h5')

In [53]:
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and digits
    text = re.sub(r'\d+',' ', text)
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(f"[{string.punctuation}]",' ',text)
    return text



In [74]:
input = 'i hate it'
text = tokenizer.texts_to_sequences([clean_text(input)])
pad_text =pad_sequences(text,maxlen=166,padding='post')
model.predict(pad_text)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


array([[0.02277315]], dtype=float32)

In [75]:
import pickle

# Assuming `tokenizer` is your tokenizer object
tokenizer_file_path = '/content/tokenizer.pkl'

# Save the tokenizer to a pickle file
with open(tokenizer_file_path, 'wb') as file:
    pickle.dump(tokenizer, file)

print(f"Tokenizer saved to {tokenizer_file_path}")


Tokenizer saved to /content/tokenizer.pkl


In [77]:
from google.colab import files

# Download the pickle file
files.download('/content/tokenizer.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [78]:
files.download('/content/sentiment_analysis_model.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [60]:
print(model)


None


In [58]:
if model is not None:
    predictions = model.predict(pad_text)
else:
    print("Model is not initialized!")


Model is not initialized!
